In [4]:
!pip install unidecode
!pip install xlsxwriter
import xlsxwriter
from unidecode import unidecode
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist
    
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 


     |████████████████████████████████| 241 kB 5.0 MB/s 
     |████████████████████████████████| 148 kB 5.0 MB/s 


Compare oldest and newest files.

Entiry Check

Adding new columns for UIP

Change to openpyxl

In [2]:
cols_ordered = ["Site Code","Site Name","EVO Site ID","Macro Region","Region","Province","Municipality","Nr. Inhabitants","Address",\
                "Altitude","Latitude","Longitude","Ground Register","Categorization by inhabitants",\
                "Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)",\
                "Categorization by Site Type","Other internal categorisation I","Categorisation by connectivity",\
                "Technology VOD","Technology VOD: Auxiliar","Fibre / Microwave","Type of Structure","Tower Height (m)",\
                "Floor space","Floor space (availability)","Status",'Billing Trigger Date',\
                "Infrastructure to be dismantled by","Date when Vodafone active equipment is removed",\
                "Infrastructure to be shared by","Core site type","Transmission Hub Site (YES/NO)",\
                "Transmission Hub Site (inc. with/without Shelters)","Transmission Site (inc. with/without Shelters)",\
                "Room configuration","An Indoor + Outdoor Site","Climate Control (YES/NO)","Air Conditioning (YES/NO)",\
                "Free-Air Cooling (YES/NO)","Power Supply ","Active/Passive DAS","Strategic Site (YES/NO)","Critical Site (YES/NO)",\
                "	If a Non-Critical and non-Transmission Hub Site, Category 1 or Category 2","Is the Site a WIP site",\
                "Indicative completion date?","Capital expenditure incurred to the MSA Effective Date",\
                "Indicative capital expenditure required to complete the site build","Vertical Infrastructure ","Site Type",\
                "Technology_VOD","Fibre/Microwave","Power Supply","# Antennas","# Minilinks",\
                "	Details of Operator Equipment installed at the Site","Site Configuration","	NOS shared site (Yes/No)",\
                "	MEO shared site (Yes/No)",\
                "Details of any Active Sharing involving Operator applicable in respect of the Site (and, where applicable, the type of Active Sharing);",\
                "	Details of the access arrangements applicable at each Site (including any access restrictions and applicable public access requirements).",\
                "Sites under Hawkins arrangements (Yes/No)","Hawkins Sites (Yes/No)","NOS Crossed Site (Yes/No)","BTS Sites (Yes/No)",\
                "Energy Provider ","POD ID","Annual Energy Consumption (kWh)","Annual Energy cost (€)",\
                "Total Energy Cost (Energy provider + Landlord)","# Lease Contracts","Countepart","Total Annual Lease (€)",\
                "Current annual lease fees (€)","Current energy lease fees (€)","Current annual other fees (€)",\
                "Annual Lease fees + Annual Other fees","Sub-Lease","(Latest) residual duration until expiring date",\
                "Maturity Cluster - Expiring date","(Latest) residual duration until expiring date after renewal",\
                "Maturity Cluster - Expiring date after renewal","Type of lease contract","Reason for no lease fee",\
                "Comments to Lease Contracts","NOS (1/0)","Total Revenues NOS (€)","Annual Hosting fee NOS (€)",\
                "Annual Energy fee NOS (€)","Annual Maintenance fee NOS (€)","Annual Other Services fee NOS (€)",\
                "Tenancy duration until expiring date NOS","Maturity Clusters NOS","MEO (1/0)","Total Revenues MEO (€)",\
                "Annual Hosting fee MEO (€)","Annual Energy fee MEO (€)","Annual Maintenance fee MEO (€)",\
                "Annual Other Services fee MEO (€)","Tenancy duration until expiring date MEO","Maturity Clusters MEO",\
                "OTMOs (1/0)","Total Revenues OTMOs (€)","Annual Hosting fee OTMOs (€)","Annual Energy fee OTMOs (€)",\
                "Annual Maintenance fee OTMOs (€)","Annual Other Services fee OTMOs (€)","Tenancy duration until expiring date OTMOs",\
                "Maturity Clusters OTMOs","Total # of 3rd Party Tenants","Name of 3rd Party Tenants","Type of Sharing",\
                "Total Hosting Fee & Services from 3rd Party Tenants (€)","Annual Hosting fee from 3rd Party Tenants (€)",\
                "Annual Energy fee from 3rd Party Tenants (€)","Annual Maintenance fee from 3rd Party Tenants (€)",\
                "Other Services fee from 3rd Party Tenants (€)","Weighted Average tenancy duration until expiring date",\
                "Total # of Tenants","Macro Cluster Tenancy","Comments to Tenant Agreements","Macro Cluster Type of contract",\
                "Easement (Servitù di passaggio)","Turistic Sites","Sites_As_Metered_Estimated","Strategic_Site_Bucket",\
                "Critical_Site_Beyond_10","First_Active_Sharing_Deployment_Type ","First_Active_Sharing_Start_Date",\
                "First_Active_Sharing_End_Date ","Subsequent_Sharing_Arrangement","Legacy_Site_Agreement_Terminated(Yes/NO)",\
                "Decommissioned Sites(True/false)"]

In [38]:
# Lendo o Ficheiro de input
def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):

    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows, engine='openpyxl')

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)

    return df

path ='/content/Vantage Towers_PT TowerDB Jun21_20210720.xlsx'
sheetname = 'Final Delivery'
skiprows = 6
skipcolumns = 1

towerdb = pd.read_excel(path, sheet_name = sheetname, skiprows=6)
towerdb = towerdb.iloc[:, 1:]
towerdb = towerdb.dropna(subset=["Site Code"], axis=0)

towerdb.rename(columns={'Billing Trigger Stop Date from May21 onwards': 'date_of_equipment_removal'}, inplace=True)
towerdb.columns = lower_str(list(towerdb.columns))
#towerdb = towerdb[lower_str(cols_ordered)]

"""First Step Check All columns received"""
path_msa = '/content/TowerDB_Portugal_20210731.csv'
df,cols_msa = csv_files(path_msa)
msa = pd.read_csv(path_msa, names=cols_msa, encoding='latin')
msa.columns = lower_str(list(msa.columns))

towerdb = towerdb[list(df_msa.columns)]

ints = ['floor space', 'tower height (m)', 'floor space (availability)', 'annual energy consumption (kwh)' ]
for i in ints:
    towerdb[i] = towerdb[i].fillna(0)
    towerdb[i] = list(map(int, towerdb[i]))
    
dates_tw = ['infrastructure ready (existing)/ to be ready (new)', 'infrastructure to be dismantled by', 'date when vodafone active equipment is removed',
            'infrastructure to be shared by', 'indicative completion date?', 'date_of_equipment_removal']
for i in dates_tw:
    towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in towerdb[i]]

errors = ['Yes; Indoor Air Conditioning and Free Air cooling / Free cooling units', 'Yes; Indoor Free Air cooling / Free cooling units']
towerdb.loc[towerdb['climate control (yes/no)'].isin(errors), 'climate control (yes/no)'] = 'Yes; Indoor Air Conditioning'

# pegar as colunas de billing
bill_cols = lower_str(['Macro Region',
             'Categorization by Site Type',
             'Categorization by Transmission Sys',
             'infrastructure ready (existing)/ to be ready (new)',
             'Core site type',             
             'Transmission Hub Site (YES/NO)',\
             'Transmission Hub Site (inc. with/without Shelters)',
             'Transmission Site (inc. with/without Shelters)',\
             'Room configuration',
             'Climate Control (YES/NO)',
             'Power Supply ',
             'Strategic Site (YES/NO)',
             'Critical Site (YES/NO)',\
             'Is the Site a WIP site',
             '	NOS shared site (Yes/No)',
             '	MEO shared site (Yes/No)',
             'BTS Sites (Yes/No)',\
             'Sites_As_Metered_Estimated',
             'Strategic_Site_Bucket',\
             'Critical_Site_Beyond_10',
             'First_Active_Sharing_Deployment_Type ',
             'Subsequent_Sharing_Arrangement',
             'Legacy_Site_Agreement_Terminated(Yes/NO)',\
             'Decommissioned Sites(True/false)'])
for col in bill_cols:
    towerdb[col] = towerdb[col].replace(['N/A', 'n/a',"0", '_',' ', np.nan,'nan'], '')

towerdb = towerdb.fillna('')

for i in ['province', 'municipality', 'address', 'site name']:
    towerdb[i] = towerdb[i].apply(unidecode)

towerdb.to_csv('/content/TowerDB_Portugal_20210831.csv', index=False)

towerdb.head(2)


,site code,site name,evo site id,macro region,region,province,municipality,nr. inhabitants,address,altitude,latitude,longitude,ground register,categorization by inhabitants,categorization by transmission sys,categorization by transmission sys (sub-cluster),categorization by site type,other internal categorisation i,categorisation by connectivity,technology vod,technology vod: auxiliar,fibre / microwave,type of structure,tower height (m),floor space,floor space (availability),status,infrastructure ready (existing)/ to be ready (new),infrastructure to be dismantled by,date when vodafone active equipment is removed,infrastructure to be shared by,core site type,transmission hub site (yes/no),transmission hub site (inc. with/without shelters),transmission site (inc. with/without shelters),room configuration,an indoor + outdoor site,climate control (yes/no),air conditioning (yes/no),free-air cooling (yes/no),...,total revenues meo (€),annual hosting fee meo (€),annual energy fee meo (€),annual maintenance fee meo (€),annual other services fee meo (€),tenancy duration until expiring date meo,maturity clusters meo,otmos (1/0),total revenues otmos (€),annual hosting fee otmos (€),annual energy fee otmos (€),annual maintenance fee otmos (€),annual other services fee otmos (€),tenancy duration until expiring date otmos,maturity clusters otmos,total # of 3rd party tenants,name of 3rd party tenants,type of sharing,total hosting fee & services from 3rd party tenants (€),annual hosting fee from 3rd party tenants (€),annual energy fee from 3rd party tenants (€),annual maintenance fee from 3rd party tenants (€),other services fee from 3rd party tenants (€),weighted average tenancy duration until expiring date,total # of tenants,macro cluster tenancy,comments to tenant agreements,macro cluster type of contract,easement (servitù di passaggio),turistic sites,sites_as_metered_estimated,strategic_site_bucket,critical_site_beyond_10,first_active_sharing_deployment_type,first_active_sharing_start_date,first_active_sharing_end_date,subsequent_sharing_arrangement,legacy_site_agreement_terminated(yes/no),decommissioned sites(true/false),date_of_equipment_removal
0,1,BRAGA,PT11000001,CONTINENT,NORTE,Braga,Braga,182679,"Travessa Escola, 4705-474 Esporoes",518,41.516600,-8.395440,,Urban,Macro,Standard,GBT,,,2G; 3G; 4G,,Fibre,Lattice,51,240,223,In Service,01/01/1992,,,,Non Core,Yes,With shelters,Non Transmission Site,Indoor,NO,Yes; Indoor Air Conditioning,Yes,No,...,0.0,0.0,0,0,0,,,0,0.0,0.0,0,0,0,,,0,,,0.0,0.0,0.0,0,0,,1,VF Only,,Leased Area - VF Only,,,Estimated Model,Non Strategic,Within 10%,,,,,No,No,
1,2,RECEZINHOS,PT13000002,CONTINENT,NORTE,Porto,Penafiel,69772,"Cruzes -Croca - Penafiel, 4560-871 Siforno",411,41.218686,-8.229177,,Urban,Macro,Standard,GBT,,,2G; 3G; 4G,,Fibre,Monopole,26,80,71,In Service,09/02/1992,,,,Non Core,No,Non Transmission Hub Site,Non Transmission Site,Indoor,NO,No,No,No,...,0.0,0.0,0,0,0,,,0,0.0,0.0,0,0,0,,,0,,,0.0,0.0,0.0,0,0,,1,VF Only,,Leased Area - VF Only,,,Estimated Model,Non Strategic,Non Critical,,,,,No,No,


In [22]:
"""Check Columns Received""" 
def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

df_col_missing = check_columns_received(towerdb, list(df_msa.columns))
df_col_missing

,Column(s) Missing


In [23]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = 'site code'
tw_doer = 'date_of_equipment_removal'
tw_dismentled = "infrastructure to be dismantled by"
tw_status = 'status'
tw_bts = 'bts sites (yes/no)'
tw_bill ='billing trigger date'
tw_wip = 'is the site a wip site'
tw_decom = 'decommissioned sites(true/false)'
tw_critical = 'critical site(yes/no)'
tw_beyond = 'critical_site_beyond_10'

msa_index ='site code'
msa_doer = 'date_of_equipment_removal'
msa_status = 'status'
msa_bts = 'bts sites (yes/no)'
msa_bill = 'billing trigger date'
msa_wip = 'is the site a wip site'
msa_decom = 'decommissioned sites(true/false)'
msa_critical = 'critical site(yes/no)'
msa_beyond = 'critical_site_beyond_10'

Condição extra para retornar no log
Only Portugal

Fazer o check da contagem de Hawkins Sites. Se 

In [26]:
def check_hawkins(df, tw_index, tw_hawkins):
    cont = len(towerdb[~(towerdb[tw_hawkins]=='No') & ~(towerdb[tw_hawkins]=='')][tw_index])
    sites = towerdb[~(towerdb[tw_hawkins]=='No') & ~(towerdb[tw_hawkins]=='')][tw_index]
    if cont != 480:
        #Retornar a coluna toda e informar que está com error pois é maior que o esperado
        df = df.loc[df[tw_index].isin(sites)]
        return df[[tw_index, tw_hawkins]]
    else:
        print('There are 480 sites.')

tw_haw = 'hawkins sites (yes/no)'
df_hawkins = check_hawkins(towerdb, tw_index, tw_haw)
df_hawkins
# Missing Hawking Sites

There are 480 sites.


In [ ]:
def check_date_columns(df, df_index,status_col,columns):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_dates = df[columns].fillna('')
    df_dates = df_dates.to_dict(orient='list')
    #df_dates['sites'] = df_dates[df_index]
    #df_dates = df_dates.set_index('sites')
    new_dic = defaultdict(list)

    date_format = re.compile(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}")

    for column in set(df_dates.keys()):
        for value in df_dates[column]:
            if date_format.match(value) == None:
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, status_col]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[status_col]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

"""Second Check Date Format dd-mm-YYYY"""
date_cols = lower_str(['infrastructure ready (existing)/ to be ready (new)', \
             'First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date '])

actives = towerdb[towerdb[tw_status]=='In Service']

"""Alterar o tipo da data para string para checkar"""
dftw_dates_errors = check_date_columns(actives, tw_index, tw_status, date_cols)
dftw_dates_errors 


In [36]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors
        
picklist_cols = lower_str(['Categorization by Transmission Sys'])

picklists = {
    'categorization by transmission sys': ['Macro','Macro+DAS', 'Public DAS','Repeater','Transmission'],
    }

#df_picklist= check_picklist(towerdb,tw_index,tw_status, picklist_cols, picklists)
df_picklist_gen= check_picklist_v1(towerdb,tw_index,tw_status, picklist_cols , picklists)
df_picklist_gen



,status
site code,


In [44]:
def check_mom_bts(df_tw, tw_index,status_col, bts_col, df_msa):

    msa_bts = df_msa[df_msa[bts_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[tw_index]]

    tw_bts = df_tw[df_tw[bts_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = df_msa[df_msa[tw_index].isin([int(i) for i in out_tower_bts])]
    filtered = filtered[[tw_index,status_col, bts_col]]
    actual = df_tw[df_tw[tw_index].isin(out_tower_bts)][[tw_index, bts_col, status_col]]
    merge = pd.merge(filtered, actual, how='inner', on=tw_index, suffixes=['_old', '_in_month'])

    return  merge[[tw_index, status_col+'_old',status_col+'_in_month', bts_col+'_old',bts_col+'_in_month']]  

actives = towerdb[towerdb[tw_status]=='In Service']
df_mom_bts = check_mom_bts(actives, tw_index,tw_status, tw_bts, msa)
df_mom_bts

,site code,status_old,status_in_month,bts sites (yes/no)_old,bts sites (yes/no)_in_month


In [35]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors
        
picklist_cols = ['Categorization by Transmission Sys', 'Core site type', 'Transmission Hub Site (YES/NO)',\
                 'Transmission Hub Site (inc. with/without Shelters)','Transmission Site (inc. with/without Shelters)',\
                 'Room configuration', 'Climate Control (YES/NO)', 'Power Supply ', 'Strategic Site (YES/NO)',\
                 'Critical Site (YES/NO)', 'Is the Site a WIP site', '	NOS shared site (Yes/No)',\
                 '	MEO shared site (Yes/No)', 'BTS Sites (Yes/No)', 'Sites_As_Metered_Estimated', 'Strategic_Site_Bucket', \
                 'Critical_Site_Beyond_10', 'Legacy_Site_Agreement_Terminated(Yes/NO)', 'Decommissioned Sites(True/false)']

picklists = {
    'categorization by transmission sys': ['Macro','Macro+DAS', 'Public DAS','Repeater','Transmission'],
    'core site type': ['Non Core', 'Case A', 'Case B'],
    'transmission hub site (yes/no)': ['Yes','No'], 
    'transmission hub site (inc. with/without shelters)': ['Non Transmission Hub Site', 'With shelters', 'Without shelters'],
    'transmission site (inc. with/without shelters)': ['Non Transmission Site', 'Without shelters'],
    'room configuration': ['Outdoor', 'Indoor'],
    'climate control (yes/no)': ['Yes; Indoor Air Conditioning','', 'No'], 
    'power supply ': ['AC','DC'],
    'strategic site (yes/no)': ['Yes','No'],
    'critical site (yes/no)': ['Yes','No'],
    'is the site a wip site': ['Yes','No'],
    '	nos shared site (yes/no)': ['Yes','No'],
    '	meo shared site (yes/no)': ['Yes','No'],
    'bts sites (yes/no)': ['Yes','No'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model'],
    'strategic_site_bucket': ['Yes - 0-5%','Non Strategic'],
    'critical_site_beyond_10': ['Beyond 10%','Within 10%','Non Critical'],
    'legacy_site_agreement_terminated(yes/no)': ['Yes','No'],
    'decommissioned sites(true/false)': ['Yes','No']}

picklist_cols = lower_str(picklist_cols)

actives = towerdb[towerdb[tw_status]=='In Service']
#df_picklist= check_picklist(towerdb,tw_index,tw_status, picklist_cols, picklists)
df_picklist= check_picklist_v1(actives,tw_index,tw_status, picklist_cols, picklists)
df_picklist
#df_picklist_errors = check_picklist_3(tw ,twdb_index, picklists)
#Tem errors



,status
site code,


In [56]:
"""New Sites Check"""
# Read msa files to match site code information from towerdb file
# Read UIS
uip = pd.read_excel('/content/UserInput_Portugal_20210831_1.xlsx', sheet_name='SiteLevel', usecols=[0,1,2], skiprows=2).fillna('')
uip_col = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip.columns = uip_col
msa_sites = [i for i in df_msa[msa_index]]
tow_sites = [str(i) for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['Site_ID']]



In [72]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col,  status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

        #create a copy to make index modifications
    """df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):"""
    return new_sites, bts_out_uis

new_sites = check_new_sites(towerdb,tw_index,tw_bts,tw_status, msa_sites, tow_sites, uip_sites)

#No one errors


In [ ]:
def check_wip(df_tw,tw_index, wip_tw, tw_bts, tw_status, df_msa):

    wip_msa = [i for i in df_msa[df_msa[wip_tw]=='Yes'][tw_index]]
    
    tw_wip_sites = [str(i) for i in df_tw[df_tw[wip_tw]=='Yes'][tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]
    
    #wip_out_tw_list = [i for i in tw_wip_sites if i not in wip_msa]
    return tw_wip_site_bts_flagged

tw_wip_site_bts_flagged= check_wip(towerdb,tw_index, tw_wip,tw_bts, tw_status, df_msa)
tw_wip_site_bts_flagged
#Tem erro no df_wip_and_bts_flagged


In [52]:
"""Decomissioned Sites Check"""
def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]

df_decom = check_decommissioned(towerdb,tw_index,tw_status, tw_decom, tw_dismentled)
df_decom
# No errors


,site code,decommissioned sites(true/false),infrastructure to be dismantled by
2508,4350,Yes,
3040,11931,Yes,


In [ ]:
"""DOER Check"""
#check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col)
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
        
    if t == 'doer':
        df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col] < current_date)|(df_tw[date_col]==''))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 
    else:

        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col]=='')]
        return filtered[[tw_index, status_col, date_col]] 

df_doer = check_tw_bill_doer(towerdb,tw_index, tw_dismentled, tw_status, '', 'doer')
df_doer
# Tem Error

,site code,status,infrastructure to be dismantled by


In [ ]:
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col, decom_col, tw_bts_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [str(i) for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]

    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]

    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files


    tw_decomiss = [str(i) for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col, decom_col]]

    #Check BTS sites
        #Check BTS sites
    bts_sites = [str(i) for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col, tw_bts_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]

    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['Critical site(beyond 10%) out of UIS'])
    critical = pd.merge(critical, filtered, how='left', left_on='Critical site(beyond 10%) out of UIS',right_on=tw_index)
    critical = critical[['Critical site(beyond 10%) out of UIS', status_tw_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
  
uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical = check_uip_tw(towerdb,tw_index,tw_status,tw_decom,tw_bts,\
                                                                                                                   'critical_site_beyond_10', uip, uip_sites)
print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_not_in_uis)
print('\n')
print(decomiss_sites_in_uip)
print('\n')
print(critical)
# NO one errors


In [67]:
def check_diffs_v2(path_current, path_last, type_file='Excel', sheet='Commercial'):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)
    type_file = type_file.lower()
    if type_file =='excel':
        _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')

        _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

        df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
        df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

        #df_final.style.apply(highlight_diff, axis=None)
        if not df_final.empty:
            return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
        else:
            print('\nNo differences Founded!\n')
            
#cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type','Input_Value_actual', 'Input_Value_before' ,'Description/Instruction', 'Frequency of Update']
df_com = check_diffs_v2( '/content/UserInput_Portugal_20210831_1.xlsx', '/content/UserInput_Portugal_20210731.xlsx')
df_com

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

df_list = [['Actives Dates Erros', dftw_dates_errors],
    ['General Picklist Errors', df_picklist_gen],
    ['BTS In month Errors', df_mom_bts],
    ['Active Picklist Errors', df_picklist],
    ['New Sites', new_sites],
    ['WIP & BTS Flagged', tw_wip_site_bts_flagged],
    ['Decom dates errors',df_decom ],
    ['UIS In Month not active', uis_sites_not_in_towerdb],
    ['TowerDB Sites out of UIS', in_service_not_in_uis],
    ['Decom Sites in UIS', decomiss_sites_in_uip],
    ['Critical Sites out UIS', critical]
    ['Commercial Differences' , df_com]]

path = '/content/towerdb_pt_errors.xlsx'
general_log_erros(df_list, path)

UIS Comparison(Old vs New)

TA Input Checks

In [5]:
# Lendo o Ficheiro de input 
path='/content/Vantage Towers_PT TowerDB Jun21_20210720.xlsx'
#test = '/content/teste_dates.xlsx'
sheet_ta = 'Tenant Agreements'
skiprows_ta = 13
skipcol = 1
ta = pd.read_excel(path, sheet_name = sheet_ta, skiprows=13)
ta = ta.iloc[:, 1:]     
ta_output_cols = ["VF Site code","VF Site name","Other operator Site code","Other operator Site name",\
                    "OWNER","TENANT","Tenant classification","Type of sharing",'Annual Hosting Fee (in €)',\
                    "M1","M2","M3","M4","M5","M6","M7","M8","M9","M10","M11","M12","Annual Energy Fee (€)",\
                    "Annual Maintenance Fee (€)","Annual Other Services Fee (€)","Total Annual Fee (€)","Authorisation Date",\
                    "Fee Starting Date","Fee Expiring Date","Renewal Option","Expiring date after renewal","Termination Date",\
                    "Terms of payments","Payment type","Index","Indexation Driver","Percentage","VAT Subject","Percentage (VAT)",\
                    "Inputs from filling responsible","Tenancy duration until expiring date (in years)","Sharing In Scope","CONTINENT"]

ta['Tenancy duration until expiring date (in years)'] = ['' for i in range(ta.shape[0])]
ta["CONTINENT"] = ['' for i in range(ta.shape[0])]
ta = ta[ta_output_cols]

dates_tw = ['Authorisation Date', 'Fee Starting Date', 'Fee Expiring Date']
for i in dates_tw:
    ta[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta[i]]

for i in ["VF Site name", "Other operator Site name"]:
    ta[i] = ta[i].fillna('')
    ta[i] = ta[i].apply(unidecode)

ta_bills = ["TENANT", "Tenant classification", "Sharing In Scope"]
for i in ta_bills:
    ta[i] = ta[i].replace(['N/A', 'n/a',"0", '-', '_',' ', np.nan,'nan'], '')

for i in ["VF Site name", "Other operator Site name"]:
    ta[i] = ta[i].fillna('')
    ta[i] = ta[i].apply(unidecode)

ta = ta.fillna('')

ta.to_csv('/content/TA_Input_Portugal_20210831.csv',encoding='windows-1252', index=False)
ta.head(2)
  

,VF Site code,VF Site name,Other operator Site code,Other operator Site name,OWNER,TENANT,Tenant classification,Type of sharing,Annual Hosting Fee (in €),M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12,Annual Energy Fee (€),Annual Maintenance Fee (€),Annual Other Services Fee (€),Total Annual Fee (€),Authorisation Date,Fee Starting Date,Fee Expiring Date,Renewal Option,Expiring date after renewal,Termination Date,Terms of payments,Payment type,Index,Indexation Driver,Percentage,VAT Subject,Percentage (VAT),Inputs from filling responsible,Tenancy duration until expiring date (in years),Sharing In Scope,CONTINENT
0,21,AVEIRO,99BL043,AVEIRO ESTE,VF,MEO,MNO,TOWER Sharing,2693.52,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,0.0,0,0,2693.52,01/03/1999,01/03/1999,,NO,,,EoP Deferred,Monthly,,,,YES,23%,,,Yes,
1,26,SAO MIGUEL DE MACHEDE,01AL016,S MIGUEL MACHEDE,VF,MEO,MNO,TOWER Sharing,2693.52,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,0.0,0,0,2693.52,01/06/2001,01/06/2001,,NO,,,EoP Deferred,Monthly,,,,YES,23%,,,Yes,


In [ ]:
import datetime as dt
#Create dates in datetime format
lista = []
for i in ta['Fee Expiring Date']:
    try:
        if i != "":
            lista.append(pd.to_datetime(i, format='%d/%m/%Y'))
        else:
            lista.append(datetime(1999, 1, 1, 0, 0, 0))
    except:
        lista.append(datetime(2199, 1, 1, 0, 0, 0))

ta['end'] = lista

ta['start'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in ta['Fee Starting Date']]
#Create diff columns to get negative values
ta['dt_dff'] = (ta['end'] - ta['start']).dt.days

start_or_end_blank = (ta['Fee Starting Date']=='')|(ta['Fee Expiring Date']=='')|(ta['Authorisation Date']=='')
negative_dates = (ta['dt_dff'] <= 0)
cols = ["VF Site code", 'Authorisation Date', 'Fee Starting Date', 'Fee Expiring Date']
ta_dates_errors = ta[start_or_end_blank|negative_dates][cols]
ta_dates_errors

In [10]:
df_class_error= ta[(ta['Tenant classification']=='')|~(ta['Tenant classification'].isin(['MNO','OTMO']))][["VF Site code", 'Tenant classification']]
df_class_error

,VF Site code,Tenant classification


In [11]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

ta_s = [['Dates Errors',ta_dates_errors],
        ['Picklist Errors', df_class_error]]

path_ta_log = '/content/TA_PT_errors.xlsx'
general_log_erros(ta_s, path_ta_log)